In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import networkx as nx

import os

In [2]:
# create adjacency matrix for network A
index_i_A, index_j_A = np.loadtxt("net1.txt", dtype=np.int, unpack=True)
dim = max(index_i_A.max(), index_j_A.max())+1
NetA_matrix = np.zeros((dim, dim))
for i,j in zip(index_i_A, index_j_A):
    # the adjacency matrix needs to be symmetric
    NetA_matrix[i, j] = 1
    NetA_matrix[j, i] = 1

# create adjacency matrix for network B
index_i_B, index_j_B = np.loadtxt("net2.txt", dtype=np.int16, unpack=True)
dim = max(index_i_B.max(), index_j_B.max())+1
NetB_matrix = np.zeros((dim, dim))
for i,j in zip(index_i_B, index_j_B):
    # the adjacency matrix needs to be symmetric
    NetB_matrix[i, j] = 1
    NetB_matrix[j, i] = 1
    
NetA = nx.from_numpy_array(NetA_matrix)
NetB = nx.from_numpy_array(NetB_matrix)

In [3]:
np.random.seed(20190410)

In [ ]:
def Gillespie(net, netName="test", lambdas=np.linspace(0.05,1, 50), mu=0.5, infects=50, n_steps=3000, N_rep=100):
    dim = net.number_of_nodes()
    # get edge list (two arrays: index_i--index_j)
    ed_list = nx.generate_edgelist(NetA, data=False)
    index_i, index_j = [], []
    for line in ed_list:
        index_i.append(int(line.split()[0]))
        index_j.append(int(line.split()[1]))
    index_i = np.array(index_i)
    index_j = np.array(index_j)
    
    lambda_mean_list=[]
    lambda_std_list=[]

    for number,l in enumerate(lambdas):
        print("***lambda =", l)
        iter_matrix=[]
        for repetition in range(N_rep):

            state = np.zeros(dim)
            # put Infected in the network
            state[np.random.choice(dim, infects, replace=False)] = 1
            I_number = [infects]

            link_matrix = np.zeros((dim,dim))
            for i,j in zip(index_i, index_j):
                if state[i] == state[j]:
                    link_matrix[i,j] = 0
                    link_matrix[j,i] = 0
                else:
                    link_matrix[i,j] = 1
                    link_matrix[j,i] = 1

            for steps in range(n_steps):
                n_I = state.sum() # #Infected
                n_S = dim - n_I   # #Susceptible
                n_N = dim
                link_sum = 0.5*link_matrix.sum()
                a1  = mu*n_I
                a2  = l*link_sum#l*n_I*n_S/N_n
                a0  = a1+a2
                p_a1= a1/a0
                # select the reaction
                # reaction 1
                if np.random.rand() < p_a1:
                    # select I at random
                    I_index    = np.where(state == 1)[0]
                    I_selected = np.random.choice(I_index)
                    #print("I_selected =", I_selected)
                    # I -> S
                    state[I_selected] = 0
                    neighbours = np.array([n for n in net.neighbors(I_selected)])
                    for Nn in neighbours:
                        if state[I_selected] == state[Nn]:
                            link_matrix[I_selected,Nn] = 0
                            link_matrix[Nn,I_selected] = 0
                        else:
                            link_matrix[I_selected,Nn] = 1
                            link_matrix[Nn,I_selected] = 1

                # reaction 2
                else:
                    # select I at random
                    I_index = np.where(state == 1)[0]
                    np.random.shuffle(I_index)

                    for k in I_index:
                        neighbours = np.array([n for n in net.neighbors(k)])
                        nn_infected = state[neighbours].sum()
                        if nn_infected == len(neighbours): 
                            continue
                        else:
                            np.random.shuffle(neighbours)
                            for NNs in neighbours:
                                if state[NNs] == 0:
                                    state[NNs] = 1

                                    NNNeighbours = np.array([n for n in net.neighbors(NNs)])
                                    for VIC in NNNeighbours:
                                        if state[NNs] == state[VIC]:
                                            link_matrix[NNs,VIC] = 0
                                            link_matrix[VIC,NNs] = 0
                                        else:
                                            link_matrix[NNs,VIC] = 1
                                            link_matrix[VIC,NNs] = 1
                                    break
                            break
                I_number.append(state.sum())  
            iter_matrix.append(I_number)
        filename = netName+"/iterations/"+str(number)+".gz"
        iter_matrix = np.array(iter_matrix)
        np.savetxt(filename, iter_matrix)
        lambda_mean_list.append(iter_matrix.mean(axis=0))
        lambda_std_list.append(iter_matrix.std(axis=0))
    np.savetxt(netName+"/mean.gz", np.array(lambda_mean_list))
    np.savetxt(netName+"/std.gz", np.array(lambda_std_list))
    return lambda_mean_list, lambda_std_list